## Importing libraries

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

In [2]:
import os
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import itertools

from sklearn.metrics import *
from sklearn.model_selection import *

import keras
from keras.layers import *
from keras.callbacks import *
from keras.models import Model
from keras import backend as K
from tensorflow.python.keras.layers import Layer
from keras import initializers, regularizers, constraints, optimizers, layers

In [3]:
# These lines of code prevent the kernal from crashing. 
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

In [4]:
X_train = pd.read_csv('/content/drive/MyDrive/movie_rating/dataset/X_train.csv')
y_train = pd.read_csv('/content/drive/MyDrive/movie_rating/dataset/y_train.csv')

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
X_train.head()

,title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,runtime,status,tagline,vote_count,credits,keywords
0,Alev Alev,Drama-Thriller,tr,NaN,0.664,Erler Film,01-01-1984,0,0,118.0,Released,NaN,2,Tarık Akan-Gülşen Bubikoğlu-Cüneyt Arkın-Çiğde...,pregnancy-model-sea captain-businessman-illega...
1,Those Who Work,Drama,fr,Frank a man of action who worked his way up al...,4.174,Box Productions-Novak Prod-Office Fédéral de l...,04-10-2018,0,0,102.0,Released,NaN,67,Olivier Gourmet-Adèle Bochatay-Delphine Bibet-...,NaN
2,Driven,NaN,en,In a world of adrenaline and speed a quadriple...,0.600,NaN,13-12-2019,0,0,17.0,Released,NaN,0,NaN,NaN
3,Netherlands Documentary,NaN,en,Sex Drugs & Other Taboo Topics The World Is To...,0.600,NaN,27-02-2020,0,0,60.0,Released,NaN,0,NaN,NaN
4,Utta Danella - Der Verlobte meiner besten Freu...,Drama,de,Katharina and Elena are best friends. After a ...,1.152,NaN,03-04-2009,0,0,88.0,Released,NaN,2,Henriette Richter-Röhl-Ina Paule Klink-Robert ...,NaN


In [7]:
X_train.drop_duplicates(inplace=True)

In [8]:
X_train.shape

(752677, 15)

In [9]:
X_train =  X_train[['overview']]

#### Here we shall work on the text which need to be tokenized and then converted to label binary endode input sequence

In [10]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 752677 entries, 0 to 752677
Data columns (total 1 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   overview  629727 non-null  object
dtypes: object(1)
memory usage: 11.5+ MB


In [11]:
X_train.isnull().sum()

overview    122950
dtype: int64

In [12]:
X_train.dropna(inplace=True)

In [13]:
X_train['overview'] = X_train['overview'].apply(lambda x: str(x).lower())

In [14]:
X_train.head()

,overview
1,frank a man of action who worked his way up al...
2,in a world of adrenaline and speed a quadriple...
3,sex drugs & other taboo topics the world is to...
4,katharina and elena are best friends. after a ...
5,with the metaphor of the stoneymollan trail a ...


In [15]:
X_train.dropna(inplace=True)

In [16]:
X_train.shape

(629727, 1)

### Building the neural network for numeric part of the code

In [17]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
# Neural Network Model for label binarizrer
X = X_train['overview']
y = y_train

In [19]:
print(X_train)

                                                 overview
1       frank a man of action who worked his way up al...
2       in a world of adrenaline and speed a quadriple...
3       sex drugs & other taboo topics the world is to...
4       katharina and elena are best friends. after a ...
5       with the metaphor of the stoneymollan trail a ...
...                                                   ...
752672  a philosophical farce about critical theory th...
752673  queens of combat presents the "heir to the thr...
752674  produced at the height of the black power move...
752676  a well behaved female prisoner on parole is se...
752677  tale of a lake is a film about the thousands o...

[629727 rows x 1 columns]


In [20]:
stop = stopwords.words('english')

In [21]:
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [22]:
X_train['overview'] = X_train['overview'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [23]:
X_train

,overview
1,frank man action worked way dedicates life wor...
2,world adrenaline speed quadriplegic race car d...
3,sex drugs & taboo topics world afraid talk
4,katharina elena best friends. year training ca...
5,metaphor stoneymollan trail hiking trail glasg...
...,...
752672,philosophical farce critical theory performati...
752673,"queens combat presents ""heir throne"" tournamen..."
752674,produced height black power movement early ‘70...
752676,well behaved female prisoner parole set return...


In [24]:
X_train['overview']


1         frank man action worked way dedicates life wor...
2         world adrenaline speed quadriplegic race car d...
3                sex drugs & taboo topics world afraid talk
4         katharina elena best friends. year training ca...
5         metaphor stoneymollan trail hiking trail glasg...
                                ...                        
752672    philosophical farce critical theory performati...
752673    queens combat presents "heir throne" tournamen...
752674    produced height black power movement early ‘70...
752676    well behaved female prisoner parole set return...
752677    tale lake film thousands lakes finland known f...
Name: overview, Length: 629727, dtype: object

### Let's tokenize the sequence 

In [25]:
vocab_size = 36000
max_length = 120

In [26]:
# this is for tokenizing the words, then each word of the tokenized list shall be given a specific index 
tokenizer = Tokenizer(num_words=vocab_size, oov_token= '<0VV>') # <0VV> is the token that the model shall replace the unseen words with. So if some word of the test case is not there in tokenized list then that word shall be replaces with 0VV this is done to maintain the size of the sentence. 
tokenizer.fit_on_texts(X_train['overview'].astype(str)) # we have to use astype(str) as there are some numbers in X_train[!]
word_index_ = tokenizer.word_index # gives index to each word and then makes a dictionary of word and their indexes.


# the numbers that would constitue to this list are those numbers which are index of the corresponding words in the dictionary.
sequence_train = tokenizer.texts_to_sequences(X_train['overview'])

In [ ]:
sequence_train

In [28]:
print(len(word_index_))

391908


In [29]:
padded_train = pad_sequences(sequence_train, maxlen = 52)

X_train = padded_train

In [ ]:
print(X_train)

In [31]:
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

### LSTM

In [37]:
model1 = tf.keras.models.Sequential()
model1.add(tf.keras.layers.Embedding(40000, 52, input_length=52)) 
model1.add(tf.keras.layers.LSTM(100, return_sequences=True,))
model1.add(tf.keras.layers.LSTM(100))
model1.add(tf.keras.layers.Dense(30, activation="relu"))
model1.add(tf.keras.layers.Dense(10, activation="relu"))
model1.add(tf.keras.layers.Dense(1, activation="linear"))

In [38]:
model1.summary()
model1.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 52, 52)            2080000   
                                                                 
 lstm (LSTM)                 (None, 52, 100)           61200     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense_3 (Dense)             (None, 30)                3030      
                                                                 
 dense_4 (Dense)             (None, 10)                310       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 2,224,951
Trainable params: 2,224,951
No

In [39]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hX_train5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [ ]:
model1.fit(X_train, y_train, epochs=500, batch_size=16, validation_split = 0.1, callbacks=callbacks_list)